In [1]:
import runpod
import time
import os
import subprocess
from dotenv import load_dotenv
import pyperclip #Copy variable to clipboard #pip install pyperclip==1.9.0

/opt/anaconda3/envs/runpod/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(os.path.join(os.path.expanduser("~"), "Desktop", "runpod_files_video", "env"))  # Get desktop path dynamically
runpod.api_key = os.getenv('RUNPOD_API_KEY_VIDEO') #Load API key from environment variable
files_path = os.path.join(os.path.expanduser("~"), "Desktop", "runpod_files_video") #Path of the files to copy to the pod
saves_path = os.path.join(os.path.expanduser("~"), "Desktop", "runpod_saves_video") #Path of the files to copy to the pod

Create Pod - Had to modify the files to take 'start_jupyter' as input, as stated in [this](https://github.com/runpod/runpod-python/pull/328/commits/071484c10438f546666667c7a2f38ad143beb435) issue

In [3]:
# Create a pod - https://github.com/runpod/runpod-python/blob/main/runpod/api/ctl_commands.py
pod = runpod.create_pod(name="test", #Set a name for the pod
                        image_name="runpod/pytorch:2.4.0-py3.11-cuda12.4.1-devel-ubuntu22.04", #Set the image to use - need python 3.11 for sam2
                        gpu_type_id="NVIDIA RTX A4500", #runpod.get_gpus() for all gpu types
                        cloud_type="COMMUNITY", #"ALL", "COMMUNITY", "SECURE"
                        support_public_ip=True, #This is the default
                        # country_code="FR", #Data should remain within the EU
                        container_disk_in_gb=30, #Set the disk size of the container
                        volume_in_gb=40, #Set the volume size of the pod
                        ports="8888/http,22/tcp", #Set the ports to expose
                        volume_mount_path="/workspace", #Set the volume mount path - Should modified below if changed here
                        start_jupyter=True, #Start the Jupyter notebook - Not exist in official documentation, found through issues.
                        )

In [4]:
# Get all my pods
pods = runpod.get_pods()
pods #Confirm the pod was created

[{'id': 'ka67yxh41r59i8',
  'containerDiskInGb': 30,
  'costPerHr': 0.19,
  'desiredStatus': 'RUNNING',
  'dockerArgs': None,
  'dockerId': None,
  'env': ['PUBLIC_KEY=ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAILXsK26MZkM32/zX6Rj6QtUWfyC4KqK1uiWfAcdyrYl6 soyrlos@gmail.com\nssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIB0TuOMKGf0P+CXwv0ft3vwHRQEfSIo4H65OdcB8x4er soyrlos@gmail.com\nssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIB0dHkhwFAWOhdpvRFCy4299gvuR9gMgvY5+swXoxjIT soyrlos@gmail.com',
   'JUPYTER_PASSWORD=8qh6y13q9ngckd2lac3y'],
  'gpuCount': 1,
  'imageName': 'runpod/pytorch:2.4.0-py3.11-cuda12.4.1-devel-ubuntu22.04',
  'lastStatusChange': 'Rented by User: Tue Jan 28 2025 09:35:33 GMT+0000 (Coordinated Universal Time)',
  'machineId': 'uxt7e4hd8wpi',
  'memoryInGb': 31,
  'name': 'test',
  'podType': 'RESERVED',
  'port': None,
  'ports': '8888/http,22/tcp',
  'uptimeSeconds': 0,
  'vcpuCount': 8,
  'volumeInGb': 40,
  'volumeMountPath': '/workspace',
  'runtime': None,
  'machine': {'gpuDisplayName': 'RT

In [5]:
time.sleep(90) #Wait for the pod to be ready and get the http url
exposed_pod = runpod.get_pods()[0]
exposed_pod

{'id': 'ka67yxh41r59i8',
 'containerDiskInGb': 30,
 'costPerHr': 0.19,
 'desiredStatus': 'RUNNING',
 'dockerArgs': None,
 'dockerId': None,
 'env': ['PUBLIC_KEY=ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAILXsK26MZkM32/zX6Rj6QtUWfyC4KqK1uiWfAcdyrYl6 soyrlos@gmail.com\nssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIB0TuOMKGf0P+CXwv0ft3vwHRQEfSIo4H65OdcB8x4er soyrlos@gmail.com\nssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIB0dHkhwFAWOhdpvRFCy4299gvuR9gMgvY5+swXoxjIT soyrlos@gmail.com',
  'JUPYTER_PASSWORD=8qh6y13q9ngckd2lac3y'],
 'gpuCount': 1,
 'imageName': 'runpod/pytorch:2.4.0-py3.11-cuda12.4.1-devel-ubuntu22.04',
 'lastStatusChange': 'Rented by User: Tue Jan 28 2025 09:35:33 GMT+0000 (Coordinated Universal Time)',
 'machineId': 'uxt7e4hd8wpi',
 'memoryInGb': 31,
 'name': 'test',
 'podType': 'RESERVED',
 'port': None,
 'ports': '8888/http,22/tcp',
 'uptimeSeconds': 0,
 'vcpuCount': 8,
 'volumeInGb': 40,
 'volumeMountPath': '/workspace',
 'runtime': {'ports': [{'ip': '100.65.15.53',
    'isIpPublic': False,
    

In [6]:
port_number=int(exposed_pod['ports'].split(',')[1].split('/')[0]) # Extract 22 from '8888/http,22/tcp'

ip, public_port = [port for port in exposed_pod['runtime']['ports'] if port['privatePort'] == port_number][0]['ip'], \
                    [port for port in exposed_pod['runtime']['ports'] if port['privatePort'] == port_number][0]['publicPort']

Connect via SSH to the Pod

In [7]:
# First connect via SSH to accept the host key
ssh_command = f"ssh -p {public_port} -i ~/.ssh/id_ed25519 -o StrictHostKeyChecking=accept-new root@{ip} exit"
subprocess.run(ssh_command, shell=True, check=True)

CompletedProcess(args='ssh -p 14465 -i ~/.ssh/id_ed25519 -o StrictHostKeyChecking=accept-new root@213.144.200.206 exit', returncode=0)

Copy files from local PC to the Pod

In [8]:
source_dir = files_path.replace('\\', '/')
scp_command = f"scp -P {public_port} -i ~/.ssh/id_ed25519 -r {source_dir}/* root@{ip}:/workspace/"

try:
    result = subprocess.run(scp_command, shell=True, check=True, capture_output=True, text=True)
    print("Successfully copied all files and folders.")
except subprocess.CalledProcessError as e:
    print(f"Error copying files: {e.stderr}")

Successfully copied all files and folders.


Create environment and install dependencies inside the Pod

In [9]:
# SSH command to connect and execute multiple commands - Takes ~3-5min to complete
ssh_commands = [
    "python -m venv /workspace/myenv",
    "source /workspace/myenv/bin/activate", 
    "cd /workspace",
    "pip install --upgrade ipykernel",
    "python -m ipykernel install --name myenv --user --display-name 'Python (myenv)'",
    "pip install -r requirements.txt && pip install flash-attn==2.6.3", #requirements.txt copied in the previous step
    "jupyter nbextension enable --py widgetsnbextension",
    "git clone https://github.com/luca-medeiros/lang-segment-anything.git",
    "cd lang-segment-anything",
    "pip install -e .",
    "pip install tensorflow[and-cuda]",
    
    #Remove print statements from lang_sam.py
    "sed -i 's/print(f\\\"Predicting {len(sam_boxes)} masks\\\")/# print(f\\\"Predicting {len(sam_boxes)} masks\\\")/g' lang_sam/lang_sam.py", 
    "sed -i 's/print(f\\\"Predicted {len(all_results)} masks\\\")/# print(f\\\"Predicted {len(all_results)} masks\\\")/g' lang_sam/lang_sam.py",

    # #To only install SAM2.1
    # "git clone https://github.com/facebookresearch/sam2.git && cd sam2",
    # "pip install -e .",
    # 'pip install -e ".[notebooks]"',
    # "wget https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_large.pt -O /workspace/sam2/checkpoints/sam2.1_hiera_large.pt",
    # "mkdir configs",
    # "mkdir configs/sam2.1",
    # "wget https://github.com/facebookresearch/sam2/blob/main/sam2/configs/sam2.1/sam2.1_hiera_l.yaml -O /workspace/sam2/configs/sam2.1/sam2.1_hiera_l.yaml",

    # # Install FFmpeg locally
    # "wget https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz -O /workspace/ffmpeg-static.tar.xz",
    # "tar -xf /workspace/ffmpeg-static.tar.xz -C /workspace",
    # "mkdir -p /workspace/bin",
    # "mv /workspace/ffmpeg-*-amd64-static/ffmpeg /workspace/bin/ffmpeg",
    # "mv /workspace/ffmpeg-*-amd64-static/ffprobe /workspace/bin/ffprobe",
    # "echo 'export PATH=\"/workspace/bin:$PATH\"' >> ~/.bashrc",
    # "source ~/.bashrc",
    # "/workspace/bin/ffmpeg -version",  # Verify FFmpeg installation
]

# Join commands with semicolons for sequential execution
command_string = "; ".join(ssh_commands)
ssh_command = f"""ssh root@{ip} -p {public_port} -i ~/.ssh/id_ed25519 "{command_string}" """

# Execute the SSH command with live output streaming
process = subprocess.Popen(ssh_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Stream output in real-time
while True:
    output = process.stdout.readline()
    if output == '' and process.poll() is not None:
        break
    if output:
        print(output.strip())

# Print any errors after completion
if process.returncode != 0:
    print("Error executing commands:")
    print(process.stderr.read())

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 7.3 MB/s eta 0:00:00
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.3 MB/s eta 0:00:00
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.6/821.6 kB 58.2 MB/s eta 0:00:00
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 25.9 MB/s eta 0:00:00
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.2/869.2 kB 90.9 MB/s eta 0:00:00
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.2/437.2 kB 65.7 MB/s eta 0:00:00
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 23.9 MB/s eta 0:00:00
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 12.2 MB/s eta 0:00:00
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 62.7 MB/s eta 0:00:00
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 95.4 MB/s eta 0:00:00
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 17.0 MB/s eta 0:00:00
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.8/387.8 kB 69.7 MB/s eta 0:00:00
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 89.9 MB/s eta 0:00:00
━━━━━

In [10]:
# Extract Jupyter password from env variables
jupyter_password = [env.split('=')[1] for env in exposed_pod['env'] if env.startswith('JUPYTER_PASSWORD')][0]
jupyter_port = exposed_pod['ports'].split("/")[0]  # Extract 8888 from "8888/http,22/tcp"
jupyter_url = "https://" + exposed_pod['id'] + "-" + jupyter_port +".proxy.runpod.net/?token=" + jupyter_password
jupyter_url

'https://ka67yxh41r59i8-8888.proxy.runpod.net/?token=8qh6y13q9ngckd2lac3y'

In [11]:
#Copy the above variable to the clipboard
pyperclip.copy(jupyter_url)
print("Jupyter URL copied to clipboard!")

Jupyter URL copied to clipboard!


Copy files from Pod to local PC

In [12]:
# # Copy output files from remote to local
# output_files_sam = "results_wave/*output*.mp4"  # Files with 'output' in their name but not 'frame' - 4mins for 600 frames+masks (remove .mp4 for that)
# # output_files_florence= "test_video/*output*"
# local_dest = saves_path

# # Ensure local destination directory exists
# os.makedirs(local_dest, exist_ok=True)

# # Construct scp command to copy files from remote to local
# scp_command_sam = f"scp -P {public_port} -i ~/.ssh/id_ed25519 root@{ip}:/workspace/{output_files_sam} {local_dest}"
# # scp_command_florence = f"scp -P {public_port} -i ~/.ssh/id_ed25519 root@{ip}:/workspace/{output_files_florence} {local_dest}"

# try:
#     result_sam = subprocess.run(scp_command_sam, shell=True, check=True, capture_output=True, text=True)
#     print(f"Successfully copied SAM output files to {local_dest}")
# except subprocess.CalledProcessError as e:
#     print(f"Error copying SAM output files: {e.stderr}")

# # try:
# #     result_florence = subprocess.run(scp_command_florence, shell=True, check=True, capture_output=True, text=True)
# #     print(f"Successfully copied Florence output files to {local_dest}")
# # except subprocess.CalledProcessError as e:
# #     print(f"Error copying Florence output files: {e.stderr}")

Terminate the Pod

In [14]:
# runpod.terminate_pod(pods[0]['id'])
# runpod.get_pods()

[]